In [102]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'02.01.2022'

In [103]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [180]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [181]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [182]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Jordan Poole,GSW,SAS,35.0,PG/SG,"5,500",41.3,13.3,49.5,76.8,MAIN,Jordan Poole,PG/SG/G/UTIL,Jordan Poole (21206992)
1,Gary Payton II,GSW,SAS,30.0,PG/SG,"3,500",33.1,13.1,41.2,76.6,MAIN,Gary Payton II,PG/SG/G/UTIL,Gary Payton II (21207233)
2,Juan Toscano-Anderson,GSW,SAS,29.0,C,"3,000",30.0,12.0,38.4,38.2,MAIN,Juan Toscano-Anderson,C/UTIL,Juan Toscano-Anderson (21207483)
3,Spencer Dinwiddie,WAS,MIL,33.5,PG,"5,900",36.5,6.9,44.6,20.2,MAIN,Spencer Dinwiddie,PG/G/UTIL,Spencer Dinwiddie (21206952)
4,Jonathan Kuminga,GSW,SAS,29.0,PF,"4,000",28.6,6.6,36.6,51.8,MAIN,Jonathan Kuminga,PF/F/UTIL,Jonathan Kuminga (21207137)


In [183]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Jordan Poole,GSW,SAS,35.0,PG/SG,5500.0,41.3,13.3,49.5,76.8,MAIN,Jordan Poole,"[PG, SG, G, UTIL]",Jordan Poole (21206992)
1,Gary Payton II,GSW,SAS,30.0,PG/SG,3500.0,33.1,13.1,41.2,76.6,MAIN,Gary Payton II,"[PG, SG, G, UTIL]",Gary Payton II (21207233)
2,Juan Toscano-Anderson,GSW,SAS,29.0,C,3000.0,30.0,12.0,38.4,38.2,MAIN,Juan Toscano-Anderson,"[C, UTIL]",Juan Toscano-Anderson (21207483)
3,Spencer Dinwiddie,WAS,MIL,33.5,PG,5900.0,36.5,6.9,44.6,20.2,MAIN,Spencer Dinwiddie,"[PG, G, UTIL]",Spencer Dinwiddie (21206952)
4,Jonathan Kuminga,GSW,SAS,29.0,PF,4000.0,28.6,6.6,36.6,51.8,MAIN,Jonathan Kuminga,"[PF, F, UTIL]",Jonathan Kuminga (21207137)


In [184]:
df.to_excel("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

# Create the Constraint Problem

Goal: Maximize DK Points

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [185]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [186]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [187]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 8

In [188]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    

prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') >= 1
prob += get_position_sum(player_vars, df, 'G') >= 3
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'F') >= 3
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 1

# prob += get_position_sum(player_vars, df, 'PG') <= 3
# prob += get_position_sum(player_vars, df, 'PG') >= 1
# prob += get_position_sum(player_vars, df, 'SG') <= 3
# prob += get_position_sum(player_vars, df, 'SG') >= 1
# prob += get_position_sum(player_vars, df, 'SF') <= 3
# prob += get_position_sum(player_vars, df, 'SF') >= 1
# prob += get_position_sum(player_vars, df, 'PF') <= 3
# prob += get_position_sum(player_vars, df, 'PF') >= 1
# prob += get_position_sum(player_vars, df, 'C') <= 3
# prob += get_position_sum(player_vars, df, 'C') >= 1

In [189]:
# total salary constraint

prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [190]:
# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [191]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [192]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['PG', 'SG', 'G', 'UTIL'] Jordan Poole GSW 5500.0 41.3
['PG', 'SG', 'G', 'UTIL'] Gary Payton II GSW 3500.0 33.1
['C', 'UTIL'] Juan Toscano-Anderson GSW 3000.0 30.0
['PG', 'G', 'UTIL'] Spencer Dinwiddie WAS 5900.0 36.5
['PF', 'F', 'UTIL'] Jonathan Kuminga GSW 4000.0 28.6
['PG', 'SG', 'G', 'UTIL'] Devin Booker PHX 9500.0 49.8
['SG', 'SF', 'F', 'G', 'UTIL'] Anthony Edwards MIN 8500.0 43.3
['PF', 'F', 'UTIL'] DeMar DeRozan CHI 9100.0 45.4


(49000.0, 308.0)